In [3]:
# Reconstructionパート

In [6]:
import CommonData.notebookutil as nbu
import sys
sys.meta_path.append(nbu.NotebookFinder())
import CommonData.tools as tools
import numpy as np
import collections
import csv
from sklearn.metrics import mean_squared_error
from matplotlib import  pyplot as plt
xaxis = range(380, 731, 10)
%matplotlib inline

In [5]:
import GlobalVariables as MyGlobalVariables

In [42]:
#色名のリストを取得
colorName = tools.load_color_name("CommonData/spectrum_24.csv")

#matrixs RGB to XYZ to RGB
#RGbとXYZの変換行列
invM_RGB = tools.mkinvM()
RGBtoXYZ = np.linalg.inv(invM_RGB)
#print invM_RGB

#等色関数
xyz1931_10nm = tools.mkxyz()
#plt.plot(xaxis, xyz1931_10nm[0])
#plt.plot(xaxis, xyz1931_10nm[1])
#plt.plot(xaxis, xyz1931_10nm[2])
#plt.show()


In [66]:
def find_clus_from_spectrum(spectrum, clusData):
    minClus = None
    minDis = 1000
    minPoint = None
    XYZ = tools.make_XYZ_from_spectrum(spectrum)
    xy = np.array([1.0*XYZ[0]/sum(XYZ), 1.0*XYZ[1]/sum(XYZ)])
    for point in clusData:
        dis = np.linalg.norm(xy - point[1:3])
        if(dis < minDis):
            minDis = dis
            minClus = point[3]
            minPoint = point
    return minClus

def find_clus_from_spectrum_fast(spectrum, kdtree, xyzfunc):
    XYZ = tools.make_XYZ_from_spectrum(spectrum, xyzfunc)
    xy = np.array([1.0*XYZ[0]/sum(XYZ), 1.0*XYZ[1]/sum(XYZ)]) 
    clus = kdtree.return_clus(xy)
    return clus

def find_clus_from_rgb(rgb, kdtree):
    XYZ = RGBtoXYZ.dot(rgb)
    xy = np.array([1.0*XYZ[0]/sum(XYZ), 1.0*XYZ[1]/sum(XYZ)]) 
    clus = kdtree.return_clus(xy)
    return clus


def load_pc_mean_multi(pcHead, meanHead, clusNum):
    pcList = collections.Counter()
    meanList = collections.Counter()
    for i in range(clusNum):
        pcList[i] = np.array(tools.loadPC(pcHead + str(i) + ".csv", showflag=0))
        meanList[i] = np.array(tools.loadSpectrum(meanHead + str(i) + ".csv", offset=0)[0])
    return pcList, meanList

def plot_data(rgbList, refSpectrum, reSpectrum, weights, invM = tools.invM):
    plt.subplots(4, 6, figsize=(16, 12))
    for i,color in enumerate(range(24)):
        #復元結果を表示
        reColor = tools.make_RGB_from_spectrum(reSpectrum[i], invM)
        print "color before",color, "after", reColor
        plt.subplot(4, 6, i)
        #print "   weights",weights[i]
        print ""
        plt.plot(xaxis, reSpectrum[i], "red")
        #正解スペクトルも表示
        plt.plot(xaxis, refSpectrum[i], "gray")
        plt.ylim(0,1)
        plt.tick_params(labelbottom='off')
        #パッチを表示
        plt.gca().add_patch(plt.Rectangle(xy=[370, 0.7], width=50, height=0.1, facecolor = np.clip(color,0,1)))    
    plt.savefig("re.png")
    plt.savefig("re.pdf")
    plt.show()
    #tools.writeResultWithHeader(reSpectrum,"hoge.csv", tools.load_color_name("CommonData/spectrum_24.csv"))
    return 0

def calc_err(refSpec, reSpec):
    errList = []
    for num,sp in enumerate(refSpec):
        tmp = mean_squared_error(reSpec[num], refSpec[num])
        errList.append(tmp)
    return errList

def reconstruct_spectrum(refSpec, pc, mean, xyzfunc, invM):
    transMatrix = invM.dot(10*xyzfunc.dot(pc))
    invTransMatrix = np.linalg.inv(transMatrix)
    color = tools.make_RGB_from_spectrum(refSpec, invM, xyzfunc)
    weight = invTransMatrix.dot(color - invM.dot(10*xyzfunc.dot(mean)))
    reSpectrum = pc.dot(weight) + mean
    return weight, reSpectrum

def reconstruct_spectra(refSpectrum, clusTable, pcList, meanList, kdtree, xyzfunc, invM):
    weights = []
    reSpectrum = []
    for num, spec in enumerate(refSpectrum):
        index = find_clus_from_spectrum_fast(spec, kdtree, xyzfunc)
        pc = pcList[index]
        mean = meanList[index]
        weight, reSpec = reconstruct_spectrum(spec, pc, mean, xyzfunc, invM)
        #transMatrix = invM.dot(10*xyzfunc.dot(pc))
        #invTransMatrix = np.linalg.inv(transMatrix)
        #color = tools.make_RGB_from_spectrum(spec, invM, xyzfunc)
        #weight = invTransMatrix.dot(color - invM.dot(10*xyzfunc.dot(mean)))
        #weights.append(weight)
        #reSpectrum.append(pc.dot(weight) + mean)
        weights.append(weight)
        reSpectrum.append(reSpec)
    return np.array(weights), np.array(reSpectrum)    

def reconstruct(pcList, meanList, testSpecFile, clusTable, nextClus, splitList, kdtree,
                         invM = tools.invM, xyzfunc = tools.xyz1931_10nm, pnum = 3, flag24 = 0):
    refSpectrum = tools.loadSpectrum(testSpecFile)
    clusNum = nextClus + 1
    rgbList = []
    for color in refSpectrum:
        rgbList.append(tools.make_RGB_from_spectrum(color, invM, xyzfunc))
    rgbList = np.array(rgbList)
    weights, reSpectrum = reconstruct_spectra(refSpectrum, clusTable, pcList, meanList, kdtree, xyzfunc, invM)

    #24色のデータでテストする場合にはグラフで表示
    if(flag24 == 1):
        plot_data(rgbList, refSpectrum, reSpectrum, weights)
        tools.writeResultWithHeader(reSpectrum,"reSpectrum.csv", range(reSpectrum.shape[0]))
    elif(flag24 == "w"):
        tools.writeResultWithHeader(reSpectrum,"reSpectrum.csv", range(reSpectrum.shape[0]))
    errList = calc_err(reSpectrum, refSpectrum)
    write_score(clusNum, errList, splitList, "EvaluateClustering" + str(clusNum) +".csv")
    return reSpectrum

def write_score(clusNum, errList, splitList, wfileName):
    wfile = open(wfileName,"ab")
    writer = csv.writer(wfile)
    writer.writerow([clusNum, np.average(errList),max(errList),min(errList),np.median(errList)] + list(splitList))
    wfile.close()

In [1]:
def reconstruct_spectrum_from_rgb(rgb, pcList, meanList, kdtree, xyzfunc, invM):
    index = find_clus_from_rgb(rgb, kdtree)
    pc = pcList[index]
    mean = meanList[index]
    print "index",index
    transMatrix = invM.dot(10*xyzfunc.dot(pc))
    invTransMatrix = np.linalg.inv(transMatrix)
    weight = invTransMatrix.dot(np.array(rgb) - invM.dot(10*xyzfunc.dot(mean)))
    reSpectrum = pc.dot(weight) + mean
    return reSpectrum